Homework 4: Neural Language Models  (& 🎃 SpOoKy 👻 authors 🧟 data) - Task 4, Option B
----

### Names
----
Names: __Katherine aristizabal, Jose Meza Llamosas__ (Write these in every notebook you submit.)

Task 4: Compare your generated sentences (25 points)
----

In this task, you'll analyze one of the models that you produced in Task 3. You'll need to compare against the corresponding file that was generated from the vanilla n-gram language model.

Choose *__one__* of option A or B (this notebook).

Option B: Evaluate the generated sentences of *word*-based models
----

Your job for this option is to measure the quality of your generated sentences for word-based models. For this option you *must* survey at least 3 people who are __not__ in this course. They need to speak and read the language that you are evaluating, but they need not be native speakers.

You will evaluate the quality of the generated sentences in the following way:
- Generate 20 sentences from your best word-based neural model. (Value of hyperparameters and n value up to you).
- Using the same level of n-gram, pair these sentences with provided sentences from the vanilla n-gram model. If you want to evaluate a model with N != 3, 4, or 5, then you'll need to train your vanilla n-gram model and generate your own comparison sentences. Ignore sentences with \<UNK\> in them for even comparison, so you'll need to over-generate to get 20.
    - Pair them (roughly) based on sentence length, so that each pair has sentences that are a roughly similar number of tokens.

Next, build a survey. For each pair of (neural LM sentence, vanilla n-ngram LM sentence), you'll ask the survey taker three binary selection questions:
1. which sentence is more grammatical?
2. which sentence makes more sense, semantically (in meaning)?
3. Overall, which sentence do you prefer?


Finally, you'll evaluate your survey results __programmatically__ (export them as a csv). Calculate the following:
1. What percentage of neural vs. vanilla n-gram LM sentences were preferred, separated along each of the three dimensions?
2. What is [Krippendorff's alpha](https://en.wikipedia.org/wiki/Krippendorff%27s_alpha) for your survey data? 

You are welcome to use a pre-built python implmenetation of the Krippendorff's alpha calculation, such as [this one](https://pypi.org/project/krippendorff/). Krippendorff's alpha is one way to measure interannotator agreement — the extent to which your survey respondants agree with one another.

You will submit your survey data (as a csv called `survey_results.csv`) __and__ your paired sentences (`paired.txt`, formatted in a way that is easy to understand) alongside this notebook.

In [1]:
 #!pip install krippendorff

In [4]:
# your imports here

import krippendorff
 
from typing import List
import numpy as np

# if you want fancy progress bars
from tqdm.autonotebook import tqdm

# Remember to restart your kernel if you change the contents of this file!
import task4_utils as nutils

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

# This function gives us nice print-outs of our models.
from torchinfo import summary

/var/folders/yc/qn_vgcrn7bl10040kgrlhnvh0000gn/T/ipykernel_18501/3465176895.py:9: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
[nltk_data] Downloading package punkt to /Users/0wner/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /Users/0wner/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
# your code here
EMBEDDINGS_SIZE = 50
NGRAM = 3
NUM_SEQUENCES_PER_BATCH = 128

TRAIN_FILE = 'spooky_author_train.csv' # The file to train your language model on
OUTPUT_WORDS = 'generated_wordbased.txt' # The file to save your generated sentences for word-based LM
OUTPUT_CHARS = 'generated_charbased.txt' # The file to save your generated sentences for char-based LM

# you can update these file names if you want to depending on how you are exploring 
# hyperparameters
EMBEDDING_SAVE_FILE_WORD = f"spooky_embedding_word_{EMBEDDINGS_SIZE}.model" # The file to save your word embeddings to
EMBEDDING_SAVE_FILE_CHAR = f"spooky_embedding_char_{EMBEDDINGS_SIZE}.model" # The file to save your char embeddings to
MODEL_FILE_WORD = f'spooky_author_model_word_{NGRAM}.pt' # The file to save your trained word-based neural LM to
MODEL_FILE_CHAR = f'spooky_author_model_char_{NGRAM}.pt' # The file to save your trained char-based neural LM to

FINAL_OUTPUT_WORDS = "final_word_generated.txt"


import zipfile
import pickle


#load previously generated sentences
#f = open(OUTPUT_WORDS, "r", encoding="utf-8", errors="ignore")
#contents = f.readlines()
with zipfile.ZipFile(FINAL_OUTPUT_WORDS, "r") as zip_ref:
    zip_ref.extractall("extracted_folder")

with open("extracted_folder/final_word_generated/data.pkl", "rb") as f:
    contents =  pickle.load(f)
#task3_list = nutils.read_file_spooky(OUTPUT_WORDS, ngram=NGRAM, char=False)
#chosen_sentences = contents[:20]
lines = contents.split('\n')
for line in lines[:20]:
    print(line)



This is really austrian sale facilis maouth recalling peering combattendo conspicuously aims madhouse.
This is really indubitably careful superinduced collated aoutside intrusive cyclops themes continuance hack.
This is really contemplations theoretical matutinal exhumation bookseller labels overlooks white servile sowing.
This is really traits determined ritual deepened motionless obstacle rèduit hooper masterminds cum.
This is really tales boccaccio disintegrated marquis illyrian subserved droughts line crawled loading.
This is really pulse attains heerd paid gregory impart ellent recusant interiorly incubus.
This is really chaldaea buzrael experts wharves thickly divulge journalists palsied extremities logically.
This is really achievements tremors purifying phipps intemperate kettles groans courtesied proceeds grope.
This is really transported donned pleases petitions gentry moods write songfully bones nest.
This is really boldest shewn resourses rufus songs frxg lend pauline altit

In [5]:
VANILLA_FILE = 'spooky_vanilla_5_word.txt' # The evaluation file of words

#sort generated sentence by sentence length
sorted_words = sorted(lines[:20], key=len)

#divide generated sentences into ngrams
ngram_sentences = []
for line in sorted_words:
    ngram_sentences.append(nutils.create_ngrams(line, n=NGRAM))
#print(ngram_sentences)

with open(VANILLA_FILE, "r") as f:
    contents_v =  f.read()

lines_v = contents_v.split('\n')

#sort them by sentence length
sorted_words_v = sorted(lines_v[:20], key=len)
#print("sorted" , nutils.format_sentence(sorted_words_v))

#divide vanilla text into ngrams
vanilla_ngrams = []

for line in sorted_words_v:
    #print(line)
    vanilla_ngrams.append(nutils.create_ngrams(line, n=NGRAM))
#print(vanilla_ngrams)

for i in range(len(sorted_words)):
    print("pair", i)
    print(sorted_words[i])
    print(sorted_words_v[i])



pair 0
This is really boldest shewn resourses rufus songs frxg lend pauline altitude gallant.
why the third degree ?
pair 1
This is really pulse attains heerd paid gregory impart ellent recusant interiorly incubus.
nor did this seem extravagant .
pair 2
This is really bye arrows vibrations caude arc abernethy ettrick blossoming ensconced took.
i began to murmur , to hesitate , to resist .
pair 3
This is really puffy dishonoured interested dazedness farce learnt flout nez thus prentice.
never a competent navigator , i could only wait .
pair 4
This is really transported donned pleases petitions gentry moods write songfully bones nest.
alas we had fallen upon the most evil of all our evil days .
pair 5
This is really dimly swirling chicago pear spectrally june ferny riot postponement mockingly.
answer me , i conjure you , with confidence and sincerity . ''
pair 6
This is really robust eyelashes archimedean steen pleiades thankfully law larnt throttling pearl.
so on the night of july , , a

Make sure that your reported results are nicely formatted!